In [1]:
import re
import numpy as np
import pandas as pd
import nltk

In [2]:
message = pd.read_csv('/content/SMSSpamCollection', sep = '\t', names = ['label','message'])

In [3]:
message.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
message['message'][0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [5]:
##text preprocessig
nltk.download('stopwords')
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
corpus = []
sbs = SnowballStemmer(language='english')
for i in range(0, len(message)):
  ##remove punctuations
  sen = re.sub('[^a-zA-Z]','',message['message'][i])
  ##remove extra white space
  sen = re.sub(' {2,}','',message['message'][i])
  ##lower case
  sen = sen.lower()
  ##split into words
  sen = sen.split();
  ##stemming and remove stop words
  sen = [sbs.stem(word) for word in sen if not word in stopwords.words('english')]
  sen = " ".join(sen)
  corpus.append(sen)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
##creating BoW (Bag of Words) model
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()
X = bow.fit_transform(corpus).toarray()

In [7]:
X[:10]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [8]:
Y = pd.get_dummies(message['label'], drop_first=True)

In [9]:
Y.head()

,spam
0,0
1,0
2,1
3,0
4,0


In [10]:
##split in train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [11]:
##train using Naive_bayes classifier
from sklearn.naive_bayes import MultinomialNB
multiNB = MultinomialNB()
multiNB.fit(X_train, Y_train)
Y_pred = multiNB.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print("Accuracy : ", accuracy_score(Y_test, Y_pred))
print("Classificatio Report : \n", classification_report(Y_test, Y_pred))
print("Confusion Matrix : \n", confusion_matrix(Y_test, Y_pred))

Accuracy :  0.9847533632286996
Classificatio Report : 
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       966
           1       0.93      0.96      0.94       149

    accuracy                           0.98      1115
   macro avg       0.96      0.97      0.97      1115
weighted avg       0.99      0.98      0.98      1115

Confusion Matrix : 
 [[955  11]
 [  6 143]]
